In [26]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [27]:
df_swimming = pd.read_csv('Olympic_Swimming_Results_1912to2020.csv')

In [28]:
df_swimming.head()

,Location,Year,Distance (in meters),Stroke,Relay?,Gender,Team,Athlete,Results,Rank
0,Tokyo,2020,100m,Backstroke,0,Men,ROC,Evgeny Rylov,51.98,1
1,Tokyo,2020,100m,Backstroke,0,Men,ROC,Kliment Kolesnikov,52,2
2,Tokyo,2020,100m,Backstroke,0,Men,USA,Ryan Murphy,52.19,3
3,Tokyo,2020,100m,Backstroke,0,Men,ITA,Thomas Ceccon,52.3,4
4,Tokyo,2020,100m,Backstroke,0,Men,CHN,Jiayu Xu,52.51,4


In [29]:
df_swimming.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4359 entries, 0 to 4358
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Location              4359 non-null   object
 1   Year                  4359 non-null   int64 
 2   Distance (in meters)  4359 non-null   object
 3   Stroke                4359 non-null   object
 4   Relay?                4359 non-null   int64 
 5   Gender                4359 non-null   object
 6   Team                  4359 non-null   object
 7   Athlete               4345 non-null   object
 8   Results               4331 non-null   object
 9   Rank                  4359 non-null   int64 
dtypes: int64(3), object(7)
memory usage: 340.7+ KB


In [30]:
# Set the display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [31]:
# Checking for NaN values in each column
nan_values = df_swimming.isna().any()
nan_values


Location                False
Year                    False
Distance (in meters)    False
Stroke                  False
Relay?                  False
Gender                  False
Team                    False
Athlete                  True
Results                  True
Rank                    False
dtype: bool

In [32]:
# Getting the rows where NaN values exist
rows_with_nan = df_swimming[df_swimming.isna().any(axis=1)]

# Printing the NaN values and corresponding rows
for column in nan_values.index:
    if nan_values[column]:
        print(f"NaN values in column '{column}':")
        print(rows_with_nan[rows_with_nan[column].isna()])
        print()

NaN values in column 'Athlete':
       Location  Year Distance (in meters)     Stroke  Relay? Gender Team  \
435         Rio  2016                4x100     Medley       1  Women  CHN   
436         Rio  2016                4x100     Medley       1  Women  CAN   
437         Rio  2016                4x100     Medley       1  Women  RUS   
438         Rio  2016                4x100     Medley       1  Women  GBR   
445         Rio  2016                4x200  Freestyle       1    Men  GER   
952     Beijing  2008                4x200  Freestyle       1    Men  USA   
1432     Sydney  2000                4x100  Freestyle       1    Men  AUS   
1467     Sydney  2000                4x200  Freestyle       1    Men  ITA   
1688    Atlanta  1996                4x100  Freestyle       1    Men  USA   
2227      Seoul  1988                4x200  Freestyle       1    Men  AUS   
2230      Seoul  1988                4x200  Freestyle       1    Men  FRA   
3625       Rome  1960                4x200  

In [33]:
#To clean the data, in the Athlete column, the NaN values will be replaced with the values written in the Team Column
df_swimming['Athlete'].fillna(df_swimming['Team'], inplace=True)
df_swimming['Results'].fillna(method='ffill', inplace=True)

In [34]:
#Check if there are NaN values
df_swimming.isna().any()

Location                False
Year                    False
Distance (in meters)    False
Stroke                  False
Relay?                  False
Gender                  False
Team                    False
Athlete                 False
Results                 False
Rank                    False
dtype: bool

In [35]:
df_swimming.dtypes

Location                object
Year                     int64
Distance (in meters)    object
Stroke                  object
Relay?                   int64
Gender                  object
Team                    object
Athlete                 object
Results                 object
Rank                     int64
dtype: object

df_swimming['Distance (in meters)'].head()

In [36]:
#Change the distance to get only the numerical value
#Change the gender by a number, 0 for men and 1 for women
df_swimming['Distance (in meters)'] = df_swimming['Distance (in meters)'].str.extract('(\d+)').astype(int)
df_swimming['Gender'] = df_swimming['Gender'].replace({'Men': 0, 'Women': 1})

In [37]:
df_swimming.head()

,Location,Year,Distance (in meters),Stroke,Relay?,Gender,Team,Athlete,Results,Rank
0,Tokyo,2020,100,Backstroke,0,0,ROC,Evgeny Rylov,51.98,1
1,Tokyo,2020,100,Backstroke,0,0,ROC,Kliment Kolesnikov,52,2
2,Tokyo,2020,100,Backstroke,0,0,USA,Ryan Murphy,52.19,3
3,Tokyo,2020,100,Backstroke,0,0,ITA,Thomas Ceccon,52.3,4
4,Tokyo,2020,100,Backstroke,0,0,CHN,Jiayu Xu,52.51,4


In [38]:
#Replace the ROC team used in 2020 Olympics by RUS, that corresponds to Russia
df_swimming['Team'] = df_swimming['Team'].replace('ROC', 'RUS')

In [39]:
df_swimming.head()

,Location,Year,Distance (in meters),Stroke,Relay?,Gender,Team,Athlete,Results,Rank
0,Tokyo,2020,100,Backstroke,0,0,RUS,Evgeny Rylov,51.98,1
1,Tokyo,2020,100,Backstroke,0,0,RUS,Kliment Kolesnikov,52,2
2,Tokyo,2020,100,Backstroke,0,0,USA,Ryan Murphy,52.19,3
3,Tokyo,2020,100,Backstroke,0,0,ITA,Thomas Ceccon,52.3,4
4,Tokyo,2020,100,Backstroke,0,0,CHN,Jiayu Xu,52.51,4


In [50]:
#Create a new column that shows the total time (Results) in seconds
# Convert 'Results' column to string type
df_swimming['Results'] = df_swimming['Results'].astype(str)

# Define a function to convert time values to seconds and hundreds of seconds
def convert_time(time_str):
    try:
        if ':' in time_str:
            # Time is in the format HH:MM:SS.SS (hours, minutes, seconds, and hundredths of a second)
            parts = time_str.split(':')
            if len(parts) == 3:
                hours, minutes, seconds = parts
                total_seconds = int(hours) * 3600 + int(minutes) * 60 + float(seconds)
            elif len(parts) == 2:
                minutes, seconds = parts
                total_seconds = int(minutes) * 60 + float(seconds)
            else:
                raise ValueError("Invalid time format")
        else:
            # Time is in the format SS.SS (seconds and hundredths of a second)
            total_seconds = float(time_str)
        return total_seconds
    except:
        # Invalid or unexpected value, assign NaN
        return np.nan

# Apply the conversion function to the 'Results' column and create a new column 'TimeInSeconds'
df_swimming['TimeInSeconds'] = df_swimming['Results'].apply(convert_time)

In [59]:
df_swimming.head(40)

,Location,Year,Distance (in meters),Stroke,Relay?,Gender,Team,Athlete,Results,Rank,TimeInSeconds
0,Tokyo,2020,100,Backstroke,0,0,RUS,Evgeny Rylov,51.98,1,51.98
1,Tokyo,2020,100,Backstroke,0,0,RUS,Kliment Kolesnikov,52,2,52.00
2,Tokyo,2020,100,Backstroke,0,0,USA,Ryan Murphy,52.19,3,52.19
3,Tokyo,2020,100,Backstroke,0,0,ITA,Thomas Ceccon,52.3,4,52.30
4,Tokyo,2020,100,Backstroke,0,0,CHN,Jiayu Xu,52.51,4,52.51
5,Tokyo,2020,100,Backstroke,0,0,ESP,Hugo Gonzalez De Oliveira,52.78,4,52.78
6,Tokyo,2020,100,Backstroke,0,0,AUS,Mitchell Larkin,52.79,4,52.79
7,Tokyo,2020,100,Backstroke,0,0,ROU,Robert Glinta,52.95,4,52.95
8,Tokyo,2020,100,Breaststroke,0,0,GBR,Adam Peaty,57.37,1,57.37
9,Tokyo,2020,100,Breaststroke,0,0,NED,Arno Kamminga,58,2,58.00


In [58]:
df_swimming['Team'].unique()

array(['RUS', 'USA', 'ITA', 'CHN', 'ESP', 'AUS', 'ROU', 'GBR', 'NED',
       'BLR', 'HUN', 'SUI', 'POL', 'GUA', 'BUL', 'FRA', 'KOR', 'UKR',
       'GER', 'AUT', 'JPN', 'FIN', 'SWE', 'RSA', 'BRA', 'LTU', 'NZL',
       'TUN', 'GRE', 'CAN', 'ISR', 'IRL', 'HKG', 'BEL', 'CZE', 'DEN',
       'KAZ', 'ISL', 'JAM', 'SGP', 'NOR', 'ZIM', 'NZ', 'NL', 'IR', 'NLD',
       'SRB', 'CUB', 'ZAF', 'TTO', 'BAH', 'VEN', 'KEN', 'PNG', 'SLO',
       'CRO', 'SVK', 'ALG', 'ARG', 'CRC', 'PUR', 'MDA', 'BAR', 'EUN',
       'SUR', 'Unified Team ', 'TCH', 'URS', 'GDR', 'FRG', 'YUG', 'SUN',
       'BGR', 'COL', 'POR', 'CHE', 'MEX', 'ECU', 'PER', 'URU', 'EGY',
       'PHI', 'BER', 'IND', 'LUX', 'ANZ'], dtype=object)